In [2]:
# -*- coding: utf-8 -*-
from bokeh.io import output_notebook
from bokeh.layouts import row
from bokeh.plotting import show

from src.libs.chart import (
    BarChart,
    BarChartConfig,
    ChartData,
    PieChart,
    PieChartConfig,
)

output_notebook()


Loading BokehJS ...

In [5]:
# 円グラフ設定
pie_config = PieChartConfig(
    title="Pizza Orders - Pie Chart",
    label_position_adjust=1.3,
)

# 棒グラフ設定
bar_config = BarChartConfig(
    title="Pizza Orders - Bar Chart",
)

# データ
data = ChartData(
    x=["Pepperoni", "Cheese", "Mixed Veggies", "Bacon"],
    y=[221, 212, 152, 72],
    colors=["red", "darkorange", "darkgreen", "hotpink"],
)


# 円グラフ
pie_chart = PieChart(data, pie_config)
pie_figure = pie_chart.render()

# 棒グラフ
bar_chart = BarChart(data, bar_config)
bar_figure = bar_chart.render()

# 並べて表示
show(row(pie_figure, bar_figure))